Project of optimization course  
Author: yunmzhang2

In [1]:
import numpy as np
import cvxpy as cp

I will use the GUROBI as the ILP solver.

In [2]:
print(cp.installed_solvers())

['ECOS', 'ECOS_BB', 'GUROBI', 'OSQP', 'SCS']


## Traffic propagation graph

![title](tpg.png)

In [3]:
E = 30 # number of edges
N = 24 # number of routers
tagn = 1 #number of tags propagates in network

The traffic propagation graph settings given as sets in implementation.

In [4]:
Edges = range(E)
pEdges = [6,7,14,15,20,21,24,25,28,29]
iEdges = []
for i in range(E):
    if i not in pEdges:
        iEdges.append(i)

Nodes = range(N)
tNodes = range(1, N)

The implementation of functions in report

In [5]:
starte = [8,17,21,0,0,0,20,19,22,11,4,9,9,9,14,15,18,16,18,13,12,10,7,23,1,3,23,2,5,6]
ende = [0,0,0,15,19,6,21,22,9,9,9,20,10,3,17,16,14,18,12,18,11,13,23,5,4,2,1,23,8,7]
ie = []
for i in range(N):
    iet = []
    for j in range(len(ende)):
        if ende[j] == i:
            iet.append(j)
    ie.append(iet)


def iE(n): # return the incoming edges of node n
    return ie[n]

def start(e):
    return starte[e]

def end(e):
    return ende

def bt(e):
    if e == 14:
        return [0]  # tag indicated by number 0
    else:
        return []
    
def at(e):
    if e == 21:
        return [0]
    else:
        return []
    
def ot(e):
    if e == 21:
        return []
    else:
        return [0]
    

In [6]:
T = np.zeros((E,tagn))

for edge in Edges:
    for tag in bt(edge):
        T[edge][tag] = 1
        
for edge in Edges:
    for tag in ot(edge):
        ans = False
        for i in iE(start(edge)):
            if T[i][tag] == 1:
                ans = True
                break
        if ans is True:
            T[edge][tag] = 1

# print(T)

In [15]:
B = np.zeros(E)
for edge in Edges:
    ans = False
    for tag in at(edge):
        if T[edge][tag] == 1:
            ans = True
            break
    if ans is True:
        B[edge] = 1

In [ ]:
def logical_and(y: cp.Variable, xs: List[cp.Variable or int or bool]):
 
    assert all([x.attributes['boolean'] for x in xs if isinstance(x, cp.Variable)] + [y.attributes['boolean']])
    n = len(xs)
    expr = sum(xs) - n * y
    return [0 <= expr, expr <= n - 1]

In [37]:
F = cp.Variable(E,boolean=True) # whether failing

R = cp.Variable(N, boolean = True) # whether reachable

A = cp.Variable(E, boolean=True)



obj = cp.Minimize(cp.sum(F))

constraints = [ R[0]==1, R[N-1]==0 ] #constraints: the connectivity of network has been broken.

for edge in iEdges:
    constraints += [ F[edge]==0 ]

# for node in tNodes:
#     for edge in iE(node):
#         constraints += [ A[edge] == cp.max(R[start(edge)] + (1-B[edge]) + (1-F[edge]) - 2, 0 )]

        
for node in tNodes:
    for edge in iE(node):
        constraints += [ A[edge] == R[start(edge)] + (1-B[edge]) + (1-F[edge])-2]

        
ieA = np.zeros((N,E))
for node in range(N):
    for edge in iE(node):
        ieA[node][edge] = 1
        

for node in tNodes:
    constraints += [ R[node] == cp.sum(A@ieA[node])]

# print(constraints)

prob = cp.Problem(obj, constraints)
prob.solve(solver=cp.GUROBI)


print(prob.status)
print(prob.value)

optimal
3.0
